In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder, MinMaxScaler

In [4]:
df = pd.read_csv('music/Song_Features.csv')

In [5]:
df.drop(labels=['samplefreq', 'channels', 'sample_points', 'audio_length_seconds'], axis=1, inplace=True)

In [6]:
df.head()

,audio_file_id,tempo_bpm,avg_diff_beat_times,std_diff_beat_times,rolloff_freq,avg_zcr,zcr_range,avg_mel_freq,std_mel_freq,avg_onset_strength,std_onset_strength,tonic,key_signature,z_dist_avg_to_tonic
0,JeffSpeed68_-_Compassion.wav,95.703125,0.6316,0.0166,7509.0,0.0907,0.651855,5.3810,40.4572,0.1577,0.1892,F,F Minor,2.4718
1,mindmapthat_-_Top_of_the_Mountain_1.wav,123.046875,0.4806,0.0155,5475.0,0.0699,0.632324,2.8197,35.5512,0.1218,0.1624,C,C Major,2.3645
2,VJ_Memes_-_Soul_Searching.wav,161.499023,0.3750,0.0155,3424.0,0.0545,0.420410,12.2590,132.4035,0.1466,0.1735,Ab,Ab Major,1.4646
3,septahelix_-_Triptych_of_Snippets.wav,89.102909,0.6664,0.0173,4028.0,0.0421,0.518066,7.2218,97.3471,0.1246,0.1804,G,G Minor,1.5473
4,destinazione_altrove_-_Kalte_Ohren_(_Remix_).wav,117.453835,0.5000,0.0116,6845.0,0.0854,0.491211,9.0604,107.0839,0.1497,0.2004,E,E Minor,2.7374


In [7]:
notes = ['C', 'C#', 'D', 'Eb', 'E', 'F', 'F#', 'G', 'Ab', 'A', 'Bb', 'B']

In [8]:
tonic_enc = OneHotEncoder(categories='auto', handle_unknown='ignore', sparse=False)
tonic = np.array(notes).reshape(-1, 1)
fitted_tonic = tonic_enc.fit(tonic)
build_tech_matrix = fitted_tonic.transform(tonic)

In [9]:
tonic_col = df["tonic"].to_numpy().reshape(-1, 1)
df = df.drop(columns=['tonic'])

In [10]:
tonic_col = fitted_tonic.transform(tonic_col)

In [11]:
notes_key = ['C Minor', 'C# Minor', 'D Minor', 'Eb Minor', 'E Minor', 'F Minor', 'F# Minor', 'G Minor', 
             'Ab Minor', 'A Minor', 'Bb Minor', 'B Minor',
            'C Major', 'C# Major', 'D Major', 'Eb Major', 'E Major', 'F Major', 
             'F# Major', 'G Major', 'Ab Major', 'A Major', 'Bb Major', 'B Major']

In [12]:
notes_key_enc = OneHotEncoder(categories='auto', handle_unknown='ignore', sparse=False)
notes_key = np.array(notes_key).reshape(-1, 1)
fitted_notes_key = notes_key_enc.fit(notes_key)

In [13]:
key_signature_col = df["key_signature"].to_numpy().reshape(-1, 1)
df = df.drop(columns=['key_signature'])

In [14]:
key_signature_col = fitted_notes_key.transform(key_signature_col)

In [15]:
scaler = MinMaxScaler(copy=True)
tempo_bpm = df['tempo_bpm'].to_numpy()
rolloff_freq = df['rolloff_freq'].to_numpy()
avg_mel_freq = df['avg_mel_freq'].to_numpy()
std_mel_freq = df['std_mel_freq'].to_numpy()
z_dist_avg_to_tonic = df['z_dist_avg_to_tonic'].to_numpy()
df = df.drop(columns=['tempo_bpm', 'rolloff_freq', 'avg_mel_freq', 'std_mel_freq', 'std_mel_freq', 'z_dist_avg_to_tonic'])

In [16]:
tempo_bpm = scaler.fit_transform(tempo_bpm.reshape(-1, 1))
rolloff_freq = scaler.fit_transform(rolloff_freq.reshape(-1, 1))
avg_mel_freq = scaler.fit_transform(avg_mel_freq.reshape(-1, 1))
std_mel_freq = scaler.fit_transform(std_mel_freq.reshape(-1, 1))
z_dist_avg_to_tonic = scaler.fit_transform(z_dist_avg_to_tonic.reshape(-1, 1))

tempo_bpm = scaler.fit_transform(tempo_bpm.reshape(-1, 1))
rolloff_freq = scaler.fit_transform(rolloff_freq.reshape(-1, 1))
avg_mel_freq = scaler.fit_transform(avg_mel_freq.reshape(-1, 1))
std_mel_freq = scaler.fit_transform(std_mel_freq.reshape(-1, 1))
z_dist_avg_to_tonic = scaler.fit_transform(z_dist_avg_to_tonic.reshape(-1, 1))

In [17]:
Y_train = df["audio_file_id"].to_numpy().reshape(-1, 1)
X_train = df.drop(columns=["audio_file_id"]).to_numpy()

In [21]:
X_train = np.hstack([X_train, tempo_bpm, rolloff_freq, avg_mel_freq, std_mel_freq, z_dist_avg_to_tonic, tonic_col, key_signature_col])

In [22]:
X_train.shape

(80, 47)

In [35]:
from annoy import AnnoyIndex

In [36]:
f = 47
t = AnnoyIndex(f, 'manhattan')

In [37]:
for i, val in enumerate(X_train):
    t.add_item(i + 1, val)

In [38]:
t.build(100)

True

In [58]:
t.save('index.tree')

True